In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog,dog_box, hidden_state, bfsp, sp) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.gru_cell = hidden_state.float().to('cuda:0')
        self.visited = 0
        self.bfsp = bfsp
        self.gru_cell_out = None
        
    def lstm_i(self, hidden_state):
        self.gru_cell = hidden_state
        self.visited = self.visited + 1

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        hidden_state = lstm_o
        self.gru_cell_out = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i(hidden_state) #((lh.detach(), lc.detach())) #DETACH

    def detach_state(self):
        self.gru_cell = self.gru_cell.detach()
            


In [6]:
class Dog:
    def __init__(self, dogid,dog_name, hidden_size, layers) -> None:
        self.dogid = dogid
        self.dog_name = dog_name
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid, box, bfsp=None, sp=None):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, box, torch.randn(self.hidden_size), bfsp, sp) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [7]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes=None):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.track_name = None
        self.raw_margins = None
        self.raw_places = None
        
        if classes!=None:
            self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def add_track_name(self, track_name):
        self.track_name = track_name


    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self, pred=False):
        if pred:
            print('pred')
        else:
            l_input = [x.gru_cell for x in self.dogs]
        return l_input

    def lstm_detach(self):
        [x.detach_state for x in self.dogs]

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_gru_output(self, hidden_states):
        for i,dog in enumerate(self.dogs):
            hs = hidden_states[i]
            #hs = hs.detach()
            dog.lstm_o(hs) #.clone())
            
    def add_weights(self, weights):
        self.weights = weights

    def add_win_weight(self):
        pass

In [8]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes=None):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid, dog_name):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid,dog_name, self.hidden_size, self.layers)

    def create_test_split(self):
        self.train_race_ids = []
        self.test_race_ids = []
        for i,r in enumerate(self.raceIDs):
            if i%10_000>(8000):
                self.test_race_ids.append(r)
            else:
                self.train_race_ids.append(r)


    def get_race_input(self, idx, Train=True) -> Race:
        if len(idx)==1:
            race = self.racesDict[self.raceIDs[idx]]
            print(f"returing race {race}")
            return race
        else:
            raceidx = operator.itemgetter(*idx)
            race_batch_id = raceidx(self.raceIDs)
            races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        
        return races #self.racesDict[raceidx]

    def get_train_input(self, idx, Train=True) -> Race:
        raceidx = operator.itemgetter(*idx)
        race_batch_id = raceidx(self.train_race_ids)
        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        return races #self.racesDict[raceidx]

    def get_test_input(self, idx, Train=True) -> Race:
        raceidx = operator.itemgetter(*idx)
        race_batch_id = raceidx(self.test_race_ids)
        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        return races #self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes
        

    def get_race_weights(self, races):
        print("weights")

    def reset_all_lstm_states(self):
        for race in self.racesDict.values:
            for dog in race.dogs:
                dog.lstmCellc = torch.rand(self.hidden_size)
                dog.lstmCellh = torch.rand(self.hidden_size)
                dog.gru_cell = torch.rand(self.hidden_size)

    def detach_all_hidden_states(self):
        for race in self.racesDict.values():
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def detach_given_hidden_states(self, races):
        for race in races:
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def create_hidden_states_dict(self):
        self.hidden_states_dict = {}
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                key = race_id+'_'+dog_id
                val = dog.gru_cell_out
                self.hidden_states_dict[key] = val

    def fill_hidden_states_from_dict(self, hidden_dict):
        for race in tqdm(self.racesDict.values()):
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                dog_prev_race_id = dog.prevrace_id
                if (dog_id=='nulldog') or (dog_prev_race_id==-1):
                    continue
                key = str(dog_prev_race_id)+'_'+dog_id
                try:
                    val = hidden_dict[key]
                    if val != None:
                        dog.gru_cell = val
                    else:
                        dog.gru_cell = torch.rand(self.hidden_size)
                except KeyError as e:
                    print(f'Key error {e}')
                    val = torch.rand(self.hidden_size)
                    dog.gru_cell = val
                    #print(f"race in = {dog.gru_cell}")
                #print(key,val)

    def to_cuda(self):
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.to(device)

    def race_prices_to_prob(self):
        for r in self.racesDict.values():
            r.implied_prob = [1/(x+0.0001) for x in r.prices]




In [9]:
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.3):
        super(GRUNet, self).__init__()
        self.gru1 = nn.GRUCell(input_size, hidden_size)
        self.gru2 = nn.GRUCell(input_size, hidden_size)
        self.gru3 = nn.GRUCell(input_size, hidden_size)
        self.gru4 = nn.GRUCell(input_size, hidden_size)
        self.gru5 = nn.GRUCell(input_size, hidden_size)
        self.gru6 = nn.GRUCell(input_size, hidden_size)
        self.gru7 = nn.GRUCell(input_size, hidden_size)
        self.gru8 = nn.GRUCell(input_size, hidden_size)
        self.rl1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.drop2 = nn.Dropout(dropout)
        self.rl2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

        self.lsftmax = nn.LogSoftmax(dim=1)

    # x represents our data
    def forward(self, race: Race)  :
        x = torch.stack([r.full_input.float() for r in race])#.detach()

        # creates list of LSTM data
        hidden_state_in = [list(i) for i in zip(*[r.lstm_input() for r in race])]

        # creates list of tensors for lstm Cells
        hCell = [torch.stack([x for x in y]) for y in hidden_state_in]

        h1 = self.gru1(x, hCell[0])
        h2 = self.gru2(x, hCell[1])
        h3 = self.gru3(x, hCell[2])
        h4 = self.gru4(x, hCell[3])
        h5 = self.gru5(x, hCell[4])
        h6 = self.gru6(x, hCell[5])
        h7 = self.gru7(x, hCell[6])
        h8 = self.gru8(x, hCell[7])

        lstm_list = [
            h1,
            h2,
            h3,
            h4,
            h5,
            h6,
            h7,
            h8,
        ]

        hCello = [i for i in zip(*[x for x in lstm_list])]
        # cCello = [i for i in zip(*[x[1] for x in lstm_list])]

        for i, r in enumerate(race):
            r.pass_gru_output(hCello[i])
        xhh = torch.cat((h1, h2, h3, h4, h5, h6, h7, h8), dim=1)
        xr1 = self.rl1(xhh)
        xd1 = self.drop1(xr1)
        xh = self.fc2(xd1)
        xd2 = self.drop2(xh)
        xr2 = self.rl2(xd2)
        xf = self.fc3(xr2)

        # output = F.softmax(xf, dim=1)
        # output = self.lsftmax(xf)
        output = xf
        return output


In [10]:
def build_pred_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])

    for i,j in tqdm(dog_stats_group):
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),-1, x['prev_race'], x['box']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=0)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED

    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])

        empty_dog_list = [null_dog_i]*8
        boxes_list = [x for x in j['box']]      
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        for n,x in enumerate(boxes_list):
            empty_dog_list[x-1] = dog_list[n]
        
        raceDB.add_race(i,trackOHE,dist)
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].add_track_name(j.track_name.iloc[0])
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    return raceDB



In [11]:
def build_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])
    dog_stats_df['runtime'] = pd.to_numeric(dog_stats_df['runtime'])

    #Generate weights for classes per track:

    grouped = dog_stats_df.groupby('track_name')
    track_weights = {}

    for i,j in grouped:
        weights = (1-(j[j['place']==1]['box'].value_counts(sort=False)/len(j[j['place']==1]))).tolist()
        if len(weights) !=8:
            weights.append(0)
        track_weights[i] = torch.tensor(weights).to(device)

    track_weights['Cranbourne'] = track_weights['Cranbourne']*5
    

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    num_features_per_dog = len(dog_stats_df['stats'][0])
    print(f"{num_features_per_dog=}")

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"]) 

    for i,j in tqdm(dog_stats_group):
        j["next_race"] = j["raceid"].shift(-1).fillna(-1)
        j["prev_race"] = j["raceid"].shift(1).fillna(-1)
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race'], x['box'], x['bfSP'], x['startprice']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=-1)
    #lsoftmax = F.log_softmax(dim=1)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(num_features_per_dog), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    #TO FIX LATER PROPER BOX PLACEMENT #FIXED
    dog_stats_df = dog_stats_df.sort_values('date')
    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])
        #margins
        empty_dog_list = [null_dog_i]*8
        empty_margin_list = [20]*8
        empty_log_margin_list = [3]*8
        empty_place_list = [8]*8
        empty_finish_list = [40]*8
        empty_price_list = [-1]*8
        untouched_margin = [20]*8

        places_list = [x for x in j["place"]]
        boxes_list = [x for x in j['box']]
        margin_list = [x for x in j["margin"]]
        time_list = [x for x in j["runtime"]]
        price_list = [x for x in j['bfSP'].astype(float)]

        # empty_log_margin_list = np.log(max(margin_list)+1)
        
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        #adjustedMargin = [margin_list[x-1] for x in boxes_list]
        for n,x in enumerate(boxes_list):
            empty_margin_list[x-1] = margin_list[n]
            empty_log_margin_list[x-1] = margin_list[n]+1
            empty_dog_list[x-1] = dog_list[n]
            empty_place_list[x-1] = places_list[n]
            empty_finish_list[x-1] = time_list[n]
            empty_price_list[x-1] = price_list[n]
            untouched_margin[x-1] = margin_list[n]
        #adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        # adjustedMargin = torch.log((torch.tensor(empty_log_margin_list).log()).to('cuda:0'))
        # adjustedMargin = F.log_softmax(torch.tensor(empty_margin_list), dim=-1).to('cuda:0')
        # adjustedMargin = softmin(torch.tensor(empty_finish_list)).to('cuda:0')
        adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        # adjustedMargin = torch.tensor(empty_place_list).to('cuda:0')
        #adjusted_dog_list = [dog_list[x-1] for x in boxes_list]
        
        raceDB.add_race(i,trackOHE,dist, adjustedMargin)
        try:
            dog_win_box = j[j['place']==1]['box'][0].item()
        except Exception as e:
            dog_win_box = 1
        raceDB.racesDict[i].win_weight = track_weights[j.track_name.iloc[0]][dog_win_box-1]
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]
        raceDB.racesDict[i].weights = track_weights[j.track_name.iloc[0]]
        raceDB.racesDict[i].raw_margins = empty_margin_list
        raceDB.racesDict[i].raw_places = empty_place_list
        raceDB.racesDict[i].untouched_margin = untouched_margin
        raceDB.racesDict[i].prices = empty_price_list

    raceDB.race_prices_to_prob()

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    raceDB.create_test_split()
    return raceDB



In [12]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    # model_name = wandb.run.name
    model_name = "test GRU saver"
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),

                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),

                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [13]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [14]:
#Testing
def validate_model(model:GRUNet,raceDB,criterion, batch_size, example_ct, epoch_loss, batch_ct):
    torch.autograd.set_detect_anomaly(True)
    sft_max = nn.Softmax(dim=-1)
    batch_size=100
    len_test = len(raceDB.test_race_ids)-batch_size
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    loss_l = []
    loss_t = []
    margins_l = []
    preds_l = []
    pred_sftmax = []
    raw_margins = []
    raw_places = []
    margins_prob = []

    prices_list = []
    raw_prices = []

    price_dict = {}
    price_dict['prices'] = []
    price_dict['imp_prob'] = []
    price_dict['pred_prob'] = []
    price_dict['pred_price'] = []
    price_dict['margin'] = []
    price_dict['onehot_win'] = []
    with torch.no_grad():
        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_test_input(races_idx)
            #tracks.extend([r.track for r in race])
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)

            _, actual = torch.max(y.data, 1)
            onehot_win = F.one_hot(actual, num_classes=8)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()

            softmax_preds = sft_max(output)
            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)

            
            loss = criterion(output, y).detach()
            loss_tensor = validation_CLE(output,y)
            loss_t.append(loss_tensor.tolist())

            loss_l.append(loss.tolist())
            preds_l.append(output.tolist())
            pred_sftmax.append(softmax_preds.tolist())
            margins_l.append(y.tolist())
            margins_prob.append(y.tolist())
            raw_margins.append([x.raw_margins for x in race])
            raw_places.append([x.raw_places for x in race])
            loss_val += loss

            price_dict['prices'].extend([x.prices for x in race])
            price_dict['imp_prob'].extend([x.implied_prob for x in race])
            price_dict['pred_prob'].extend(softmax_preds.tolist())
            price_dict['pred_price'].extend([(1/(x)).tolist() for x in softmax_preds])
            price_dict['margin'].extend([x.raw_margins for x in race])
            price_dict['onehot_win'].extend(onehot_win.tolist())

            

        loss_list = []

        #print("start loss calc")
        for i,l in enumerate(loss_l):
            for j in range(0,7):
                loss_list.append([preds_l[i][j],margins_l[i][j],loss_t[i][j],l[j],pred_sftmax[i][j],margins_prob[i][j], raw_margins[i][j], raw_places[i][j]])

    loss_df = pd.DataFrame(loss_list, columns=['Preds','Margins','Indiviual Losses','Losses','softmaxPreds','Softmax Margins','Raw margins', 'Raw places'])
    loss_table = wandb.Table(dataframe=loss_df)


    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })
    
    logdf['correct'] = logdf.apply(lambda x: 1 if x['actuals']==x['preds'] else 0, axis=1)
    logdf['profit'] = logdf.apply(lambda x: 0 if x['bfsps']<1 else x['bfsps']-1  if x['correct'] else -1, axis=1)
    logdf.to_csv('logDFtest.csv')

    table = wandb.Table(dataframe=logdf)

    logdf['count'] = 1
    logdf['eligible_ct'] = logdf.apply(lambda x: 1 if x['bfsps'] > 1 else 0, axis = 1)

    track_df = logdf.groupby('track', as_index=False).sum().reset_index()
    track_table = wandb.Table(dataframe=track_df)
    
    prices_df = pd.DataFrame(price_dict)

    prices_df['sum_price'] = prices_df.apply(lambda x: sum(x['prices']), axis = 1)
    prices_df = prices_df[prices_df['sum_price']>0]

    prices_flat = [item for sublist in prices_df['prices'].tolist() for item in sublist]
    pred_prices = [item for sublist in prices_df['pred_price'].tolist() for item in sublist]
    onehot_win  = [item for sublist in prices_df['onehot_win'].tolist() for item in sublist]

    all_price_df = pd.DataFrame(data={'prices':prices_flat, 'pred_price':pred_prices, 'onehot_win':onehot_win})
    all_price_df = all_price_df[all_price_df['prices']>1]
    all_price_df['imp_prob'] =  all_price_df.apply(lambda x: 1/x['prices'], axis = 1)
    all_price_df['pred_prob'] =  all_price_df.apply(lambda x: 1/x['pred_price'], axis = 1)
    all_price_df['bet amount'] = all_price_df.apply(lambda x: (x['pred_prob']-x['imp_prob'])*100 if (x['pred_prob']>x['imp_prob'])&(1>x['imp_prob']>0) else 0, axis = 1)
    all_price_df['profit'] = all_price_df.apply(lambda x: x['bet amount']*(x['prices']-1)*0.95 if x['onehot_win'] else -1*x['bet amount'], axis = 1)
    all_price_df['colour'] = all_price_df.apply(lambda x: "profit" if x['profit']>0 else ("loss" if x['profit']<0 else "no bet"), axis=1)
    price_table = wandb.Table(dataframe=prices_df)

    
    
    try:
        wandb.log({"table_key": table,"loss_table": loss_table,"track_df": track_table,"price_table":price_table, 'allprice_df':all_price_df })
    except Exception as e:
        print(e)

    print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": torch.mean(loss_val)/len_test, "correct": correct, 'profit': logdf[logdf['actuals']==logdf['preds']]['profit'].sum(), 'multibet profit':all_price_df['profit'].sum(), 'multibet outlay':all_price_df['bet amount'].sum() })

    return correct/total

In [15]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)

In [16]:
def train(model, raceDB:Races, criterion, optimizer,scheduler, config=None):
    torch.autograd.set_detect_anomaly(True)

    last = 0
    batch_size = 100
    len_train = len(raceDB.train_race_ids)-batch_size
    example_ct = 0  # number of examples seen
    batch_ct = 0
    m = nn.LogSoftmax(dim=1)
    epoch_loss=0
    batch_before_backwards = 10

    #losses = torch.tesnor()
    for epoch in trange(2000):
        model.train()
        batch_ct = 0
        setup_loss = 1
        batch_size= randint(25,2000)
        batch_before_backwards = randint(5,20)
        wandb.log({"batch_size":batch_size, 'batch_before_backwards':batch_before_backwards})
        for i in trange(0,len_train,batch_size, leave=False):
            last = i
            #print(f"{i=}\n{batch_ct=}, {setup_loss=}, {batch_ct+1%10==0=}")
            if ((batch_ct+1)%batch_before_backwards==0):
                if setup_loss:
                    print("hit here")
                    continue
                else:
                    epoch_loss = torch.mean(epoch_loss)
                    optimizer.zero_grad()
                    epoch_loss.backward()
                    optimizer.step()
                    model.zero_grad()
                    raceDB.detach_all_hidden_states()
                    setup_loss = 1
                    wandb.log({"batch_loss": epoch_loss.item(), "batch_before_backwards": batch_before_backwards}, step = example_ct)
                    

            batch_ct += 1

            races_idx = range(last,last+batch_size)
            try:
                race = raceDB.get_train_input(races_idx)
            except Exception as e:
                continue
            X = race
            y = torch.stack([x.classes for x in race])
            w = torch.stack([x.win_weight for x in race])
            _, actual = torch.max(y.data, 1)
            output = model(X)

            
            example_ct +=  batch_size

            loss = criterion(output, y)*w
            if setup_loss:
                epoch_loss = loss
                setup_loss=0

            if ((batch_ct + 1) % 25) == 0:                    
                    train_log(torch.mean(loss), example_ct, epoch)

            #epoch_loss = torch.stack([epoch_loss, loss])
            epoch_loss = epoch_loss + loss
            

        print("finished epoch")
        setup_loss = 1

        wandb.log({"epoch_loss": torch.mean(epoch_loss)}, step = example_ct)
        acc = validate_model(model,raceDB,criterion, 8, example_ct, epoch_loss, batch_ct)
        epoch_loss = 0  
        print(acc)
        scheduler.step(acc)
    #print(losses)
    return model



In [17]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df_FASTTRACK new margins.npy', 'rb')
hidden_size = 64
raceDB = build_dataset(dog_stats_file, hidden_size)

num_features_per_dog=16


  0%|          | 0/38032 [00:00<?, ?it/s]

  0%|          | 0/143082 [00:00<?, ?it/s]

number of races = 143082, number of unique dogs = 38032


In [18]:

#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
hidden_size = 64
predDB = build_pred_dataset(dog_stats_file, hidden_size)

  0%|          | 0/363 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

number of races = 48, number of unique dogs = 363


In [19]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)
# len(raceDB.hidden_states_dict)
# predDB.to_cuda()

In [20]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [21]:
print(os.getcwd())
#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\dog_stats_df_FASTTRACK new margins.npy", 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)
dog_stats_df = dog_stats_df.sort_values('date')
dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])
dog_stats_df
weights = (1-(dog_stats_df[dog_stats_df['place']==1]['box'].value_counts(sort=False)/len(dog_stats_df[dog_stats_df['place']==1]))).tolist()
weights = torch.tensor(weights).to(device)
weights

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


tensor([0.8130, 0.8508, 0.8918, 0.8742, 0.9023, 0.9040, 0.8893, 0.8746],
       device='cuda:0')

In [22]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      if prev_model:
        model = prev_model
      else:
        model = GRUNet(203,config['hidden_size'])
        if model_state_dict:
          model.load_state_dict(model_state_dict)

            

      criterion = nn.CrossEntropyLoss(reduction='none', label_smoothing=0.5, )
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',threshold=0.001, patience=10, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      # if sweep:
    #   raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset)

In [23]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}

In [24]:
raceDB.detach_all_hidden_states()

In [25]:
raceDB.create_test_split()

In [26]:
print(raceDB)

In [27]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": 70000},
    "hidden_size": {"value":16}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [500,1000,5000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [500, 1000, 5000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 16},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 0.00011},
                'len_data': {'value': 70000},
                'loss': {'values': ['L1']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 70000},
  'hidden_size': {'value': 16},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['L1']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 0.00011, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [500, 1000, 5000]}}}

In [28]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

In [29]:
save_path = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\test GRU saver\test GRU saver_200.pt"
saved_dict = torch.load(save_path)
model_state_dict = saved_dict['model_state_dict']
raceDB.fill_hidden_states_from_dict(saved_dict['db'])
raceDB.to_cuda()

  0%|          | 0/143082 [00:00<?, ?it/s]

In [30]:
(model,dataset) = model_pipeline(raceDB,config=wandb_config_static, model_state_dict=model_state_dict)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(203, 64)
  (gru2): GRUCell(203, 64)
  (gru3): GRUCell(203, 64)
  (gru4): GRUCell(203, 64)
  (gru5): GRUCell(203, 64)
  (gru6): GRUCell(203, 64)
  (gru7): GRUCell(203, 64)
  (gru8): GRUCell(203, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (lsftmax): LogSoftmax(dim=1)
)


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

finished epoch


...

finished epoch


  0%|          | 0/279 [00:00<?, ?it/s]

accuray: 0.2982795698924731
0.2982795698924731


  0%|          | 0/62 [00:00<?, ?it/s]

finished Early



In [ ]:
(model,dataset) = model_pipeline(raceDB,config=wandb_config_static, prev_model=model)

{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(203, 64)
  (gru2): GRUCell(203, 64)
  (gru3): GRUCell(203, 64)
  (gru4): GRUCell(203, 64)
  (gru5): GRUCell(203, 64)
  (gru6): GRUCell(203, 64)
  (gru7): GRUCell(203, 64)
  (gru8): GRUCell(203, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (lsftmax): LogSoftmax(dim=1)
)


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/1150 [00:00<?, ?it/s]

finished epoch


  0%|          | 0/279 [00:00<?, ?it/s]

...

  0%|          | 0/279 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp3cx_y2kvwandb-media\\3ic9vl5t.table.json'
accuray: 0.29405017921146953
0.29405017921146953


  0%|          | 0/1150 [00:00<?, ?it/s]

finished Early



accuracy,▅▅███▁██▄▇
batch_loss,▅█▅▁▅█▅▁▅▅▅▁▅▅▆▁▅▅▆▁▅▅▆▁█▅▆▁█▅▆▁█▅▁▁█▅▁▅
correct,▅▅███▁██▄▇
epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇█
epoch_loss,▆▅█▇▆▁▅▅▄▄
loss,▇▅▇▁▁▂█▁▇▂▁▁▁▁▂▁▂▁▂▁▂█▁▁▅▇▁▁▂█▁▇▂▁▁▁▂▂▁▂
loss_val,▅█▂▇▄█▂▄▇▁
profit,▅▅▇█▇▁█▆▄▆
accuracy,0.29405
batch_loss,77.28403
correct,8204


In [ ]:
raceDB.hidden_states_dict

{'498695480_296096016': tensor([-1.0000,  1.0500, -0.6879, -1.2569, -1.1016,  0.8615, -0.0183,  0.9999,
         -1.0000, -0.3919, -2.8866, -1.0000, -1.6032, -0.9819,  1.0000, -0.4281,
          1.2264, -1.0000,  1.0000,  1.0000, -0.1428, -1.8032, -0.5396, -0.0531,
          0.9999,  0.1418, -0.9999, -0.3113,  1.0000,  0.8645,  0.1349,  0.9997,
         -0.0242, -1.3021, -0.2483,  0.8888, -0.9998,  0.4114, -0.7088,  0.1655,
         -1.0000, -0.3554, -1.0000, -1.0000, -0.7610, -0.6507, -1.4607, -0.0987,
          1.0000, -0.9806,  1.0000, -0.2016, -1.0000, -0.9996, -1.0000, -1.2254,
          0.4229, -1.0002, -1.0000, -1.0000, -1.2049, -1.0000,  1.6590, -0.9637],
        device='cuda:0', grad_fn=<UnbindBackward0>),
 '498695480_314367487': tensor([ 0.9759, -1.0940, -0.7505,  1.0000, -1.0000, -1.0000,  0.9995,  0.3528,
          0.2053, -0.6713, -0.9301, -1.0000, -1.8133, -1.2187, -0.7992,  0.8279,
          1.0000,  1.0000,  1.0000, -0.1702,  0.0343,  1.3304,  0.0514, -0.3913,
         

In [ ]:

dataset.create_hidden_states_dict()

In [ ]:
model_saver(model, "None", 200, 0.1, dataset.hidden_states_dict)

created path


In [ ]:
predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)